# set up

In [32]:
import time

In [16]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.vectorstores.neo4j_vector import Neo4jVector

In [9]:
import Vector_database_functions as db
import ChatTestDB as log
import Harry_Potter_questions as HP

# Embeddings

In [10]:
model_name = "sentence-transformers/all-mpnet-base-v2"
# model_name = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

In [11]:
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [12]:
# Get Data

In [13]:
vectordb = db.get_db( "Harry_Potter", "data/Chroma/", embeddings)

In [14]:
url = "bolt://localhost:7687"
username = "neo4j"
password = "pleaseletmein"

In [17]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    url=url,
    username=username,
    password=password,
    index_name='persons',
    node_label=["Person", 'Location', 'Skill', 'Organization', 'Award', "Country", 'Religion' ],
    text_node_properties=['name', "positionHeld", "causeOfDeath", "dateOfBirth", "numberOfChildren", "academicDegree", "dateOfDeath", "age", "productType", "foundingDate" ],
    embedding_node_property='embedding',
)

# LLM

In [18]:
llm = ChatOllama(model="llama3")

In [19]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use five sentences maximum. Keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 

{context}

Question: {question}

Helpful Answer:
"""

In [20]:
prompt = PromptTemplate.from_template(template)

In [27]:
question = HP.question1

In [22]:
vector_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_index.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [23]:
chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

In [28]:
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})

In [29]:
result["result"]

"According to the LEGO Harry Potter: Characters of the Magical World, Harry Potter's wand is not specified in this context. The information provided only lists the characters' names and does not mention their wands.\n\nThanks for asking!"

# Test of awnsers

In [73]:
question = HP.question1
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

According to the LEGO Harry Potter series, Harry Potter's wand is an important part of his magical abilities. The answer is not explicitly stated in this context, but it can be found in the original book series by J.K. Rowling. 

Thanks for asking!
4.677424649397532


In [74]:
question = HP.question2
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

Harry Potter belongs to Gryffindor House. Thanks for asking!
3.3670823852221172


In [75]:
question = HP.question3
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

Merope Gaunt was Bellatrix Lestrange's mother and the sister of Marvolo Gaunt. She was a witch who became obsessed with Tom Riddle (Lord Voldemort) and had an affair with him, resulting in Harry Potter's father, James Potter. Thanks for asking!
2.2374295552571613


In [76]:
question = HP.question4
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

I don't have enough information to answer this question. Thanks for asking!
1.40389830271403


In [77]:
question = HP.question5
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

Fred and his friends in Gryffindor House created the Marauder's Map. Thanks for asking!
1.773429799079895


In [78]:
question = HP.question6
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

Based on the context, I can tell you that the Half-Blood Prince refers to Severus Snape. He is a character in the Harry Potter series and is revealed to be a half-blood, meaning he has both pure-blood and non-pure-blood ancestry. Thanks for asking!
1.9021825393040974


In [79]:
question = HP.question7
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

I don't know. Thanks for asking!
1.8051441510518391


In [80]:
question = HP.question8
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

I don't have that information. Thanks for asking!
3.4496345202128094


In [81]:
question = HP.question9
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

Voldemort created a total of 7 Horcruxes. Thanks for asking!
3.3335898796717327


In [82]:
question = HP.question10
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

There are three Deathly Hallows. Thanks for asking!
3.5062723755836487


In [83]:
question = HP.question11
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

The entrance to the Chamber of Secrets is located in the second floor bathroom at Hogwarts Castle. Thanks for asking!
2.314288886388143


In [84]:
question = HP.question12
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

The answer to your question is Dobby. Thanks for asking!
1.5309942523638407


In [85]:
question = HP.question13
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

I don't know. The text doesn't mention this specific goblin helping Harry break into Gringotts. Thanks for asking!
2.8835273067156475


In [86]:
question = HP.question14
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

According to the context, Sirius Black is Harry Potter's maternal first cousin, once removed. This means that Sirius was Harry's mother's first cousin, but they are also related through marriage. Thanks for asking!
1.618551222483317


In [87]:
question = HP.question15
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

The three-headed dog that guards the Philosopher's Stone is named Fluffy. Thanks for asking!
3.371462380886078


In [88]:
question = HP.question16
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

According to the Harry Potter Wiki, the hippogriff that Harry saves from execution is named Buckbeak. Thanks for asking!
3.3960209250450135


In [89]:
question = HP.question17
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

According to the context, the ghost of Ravenclaw house is Helena Ravenclaw. Thanks for asking!
2.5321393171946207


In [90]:
question = HP.question18
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

Scabbers is the name of Ron's pet rat. Scabbers is an old, grey rat that misses a toe and was inherited by Ron from Percy. Thanks for asking!
2.421801781654358


In [91]:
question = HP.question19
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

There are seven Weasley siblings: Bill, Charlie, Fred, George, Ginny, Molly (the twins), and Percy. Thanks for asking!
2.5810751080513


In [92]:
question = HP.question20
start = time.time()
result = chain.invoke({"query": question, "context": vector_qa.invoke(question)})
end = time.time()
times = (end - start)/60
log.LogChat(4, "Vector_with_graph_as_context", question, result['result'], times)
print(result["result"])
print(times)

The Weasley family has seven children, and the youngest sibling is Ginny Weasley.
2.5281138022740683
